# Climate Change Modeling

In [ ]:
# 03 — Projections & Scenarios
import pandas as pd, numpy as np, pickle, os
from sklearn.preprocessing import StandardScaler

ART_DIR = "../artifacts"
MODEL_PTH = os.path.join(ART_DIR, "rf_model.pkl")
SCALER_PTH = os.path.join(ART_DIR, "rf_scaler.pkl")

with open(MODEL_PTH, "rb") as f:
    model = pickle.load(f)
with open(SCALER_PTH, "rb") as f:
    scaler: StandardScaler = pickle.load(f)

# Load future data (edit path)
FUTURE_PATH = "../data/future_climate_data.csv"  # create this with your scenarios
future = pd.read_csv(FUTURE_PATH)

# Ensure numeric features used in training are present
num_cols = future.select_dtypes("number").columns.tolist()
X_future = scaler.transform(future[num_cols])
preds = model.predict(X_future)
future["prediction"] = preds
future.head()

In [ ]:
# Save projections
out_path = "../reports/projections.csv"
future.to_csv(out_path, index=False)
print("Saved:", out_path)